In [ ]:
import tensorflow as tf
import numpy as np

# ✅ Load your original Keras model (.h5 or SavedModel)
model_path = r"D:\project\red_green_classifier.h5"  # Update this if it's a SavedModel directory
model = tf.keras.models.load_model(model_path)

# ✅ Create a converter object
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# ✅ Enable full INT8 quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# ✅ Representative dataset function for quantization (important!)
def representative_dataset():
    for _ in range(100):
        data = np.random.rand(1, *model.input_shape[1:]).astype(np.float32)
        yield [data]

converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# ✅ Convert and save the model
quantized_tflite_model = converter.convert()
with open("model_int8.tflite", "wb") as f:
    f.write(quantized_tflite_model)

print("✅ Successfully converted to model_int8.tflite")


In [11]:
import numpy as np

# Read the quantized TFLite model
tflite_model_path = r"model_int8.tflite"
with open(tflite_model_path, "rb") as f:
    tflite_model = f.read()

# Convert model into C array
hex_array = ", ".join(f"0x{b:02x}" for b in tflite_model)

# Generate C header file
header_content = f"""\
#ifndef MODEL_H
#define MODEL_H

#include <cstdint>

alignas(8) const unsigned char g_model[] = {{
    {hex_array}
}};

const int g_model_len = {len(tflite_model)};

#endif  // MODEL_H
"""

# Save to model.h
with open("model.h", "w") as f:
    f.write(header_content)

print("✅ model.h file created successfully!")


✅ model.h file created successfully!


In [13]:
import tensorflow.lite as tflite
import numpy as np

# Load your TFLite model
interpreter = tflite.Interpreter(model_path=r"model_int8.tflite")

# Allocate tensors
interpreter.allocate_tensors()

# Calculate the required tensor memory
tensor_details = interpreter.get_tensor_details()
tensor_arena_size = sum(np.prod(tensor['shape']) * np.dtype(tensor['dtype']).itemsize for tensor in tensor_details)

print(f"🔹 Estimated Tensor Arena Size: {tensor_arena_size} bytes")


🔹 Estimated Tensor Arena Size: 1871054 bytes
